<a href="https://colab.research.google.com/github/Jalam1001/huggingface/blob/main/Another_copy_of_LFM2_MoE_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq git+https://github.com/huggingface/transformers.git@0c9a72e4576fe4c84077f066e585129c97bfd4e6 --progress-bar off

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_id = "LiquidAI/LFM2-8B-A1B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="bfloat16",
#    attn_implementation="flash_attention_2" <- uncomment on compatible GPU
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Generate answer
prompt = "What is C. elegans?"
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True,
).to(model.device)

output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.3,
    min_p=0.15,
    repetition_penalty=1.05,
    max_new_tokens=512,
)

print(tokenizer.decode(output[0], skip_special_tokens=False))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.36G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

<|startoftext|><|im_start|>user
What is C. elegans?<|im_end|>
<|im_start|>assistant
**C. elegans**, short for *Caenorhabditis elegans*, is a small, transparent nematode (roundworm) that has become one of the most important model organisms in biological research. Here’s a detailed breakdown:

### Key Features:
1. **Biological Characteristics**:
   - **Size**: About 1 millimeter long, visible to the naked eye but easily studied under a microscope.
   - **Transparency**: Its body is entirely see-through, allowing direct observation of internal processes like cell development and movement.
   - **Simple Anatomy**: Comprises exactly **959 somatic cells** (in adults), making it ideal for mapping developmental biology and gene function.

2. **Genetic Simplicity**:
   - A fully sequenced genome (~100 million base pairs) with just **~20,000 genes**, many of which are conserved in humans (e.g., genes involved in development, neurodegeneration, and aging).
   - Easy to manipulate genetically—muta

In [ ]:
from textwrap import fill
from IPython.display import Markdown, display

def run_chat(prompt, system_prompt="You are a helpful assistant.",
             max_new_tokens=512, temperature=0.3):
    """
    Wrapper for generating structured chat with formatted output.
    """
    # Build messages with system + user
    messages = [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": prompt.strip()},
    ]

    # Encode using chat template
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        tokenize=True,
    ).to(model.device)

    # Generate
    output = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        min_p=0.15,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )

    # Decode and clean
    decoded = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Nicely show markdown in Colab
    display(Markdown(f"**User:** {prompt}\n\n**Assistant:**\n\n{decoded}"))

    return decoded

# Example usage:
run_chat("Explain edge computing and give one practical example.")

**User:** Explain edge computing and give one practical example.

**Assistant:**

system
You are a helpful assistant.
user
Explain edge computing and give one practical example.
assistant
Edge computing is a distributed computing paradigm that brings data processing and storage closer to the location where data is generated, rather than relying solely on centralized cloud servers. This approach reduces latency, conserves bandwidth, and enhances real-time decision-making by processing data locally at the "edge" of the network—such as on devices, gateways, or local servers.

**Practical Example:**  
In a smart factory, industrial sensors monitor machinery performance in real time. Instead of sending all sensor data to a distant cloud server for analysis, edge computing allows data processing to occur directly on local edge devices (e.g., industrial gateways or on-premises servers). These devices analyze the data instantly to detect anomalies, predict equipment failures, or trigger immediate adjustments—such as slowing a machine to prevent damage. This reduces response time from seconds to milliseconds, improving efficiency and safety without overwhelming network bandwidth.

'system\nYou are a helpful assistant.\nuser\nExplain edge computing and give one practical example.\nassistant\nEdge computing is a distributed computing paradigm that brings data processing and storage closer to the location where data is generated, rather than relying solely on centralized cloud servers. This approach reduces latency, conserves bandwidth, and enhances real-time decision-making by processing data locally at the "edge" of the network—such as on devices, gateways, or local servers.\n\n**Practical Example:**  \nIn a smart factory, industrial sensors monitor machinery performance in real time. Instead of sending all sensor data to a distant cloud server for analysis, edge computing allows data processing to occur directly on local edge devices (e.g., industrial gateways or on-premises servers). These devices analyze the data instantly to detect anomalies, predict equipment failures, or trigger immediate adjustments—such as slowing a machine to prevent damage. This reduces

In [ ]:
from textwrap import fill
from IPython.display import Markdown, display

def run_chat(prompt, system_prompt="You are a helpful assistant.",
             max_new_tokens=512, temperature=0.3, wrap_width=100):
    """
    Wrapper for generating structured chat with clean, readable output.
    """
    # Build messages
    messages = [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": prompt.strip()},
    ]

    # Encode using chat template
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        tokenize=True,
    ).to(model.device)

    # Generate output
    output = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        min_p=0.15,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )

    # Decode
    decoded = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Strip off prepended roles (system/user)
    if "assistant" in decoded.lower():
        decoded = decoded.split("assistant", 1)[-1].strip(": \n")

    # If text looks like markdown → render with Markdown
    if any(x in decoded for x in ["#", "*", "-", "`"]):
        display(Markdown(f"**User:** {prompt}\n\n**Assistant:**\n\n{decoded}"))
    else:
        # Wrap long plain text lines for readability
        wrapped = fill(decoded, width=wrap_width)
        display(Markdown(f"**User:** {prompt}\n\n**Assistant:**\n\n{wrapped}"))

    return decoded

In [ ]:
from textwrap import fill
from IPython.display import Markdown, display

def run_chat(prompt,
             system_prompt="You are a helpful assistant.",
             max_new_tokens=512,
             temperature=0.3,
             wrap_width=100,
             return_text=False):
    """
    Clean chat wrapper for LFM2 in Colab.
    Shows only the assistant reply; no prompt echo, no duplicate output.
    """
    # Build messages
    messages = [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user",   "content": prompt.strip()},
    ]

    # Encode with chat template
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        tokenize=True,
    ).to(model.device)

    # Generate
    output = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        min_p=0.15,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )

    # 🔑 Decode only *new* tokens (no prompt echo)
    gen_tokens = output[0, input_ids.shape[-1]:]
    text = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    # Render nicely (markdown if present, else wrapped plain text)
    if any(x in text for x in ("#", "* ", "- ", "`", "\n\n")):
        md = f"**User:** {prompt}\n\n**Assistant:**\n\n{text}"
    else:
        md = f"**User:** {prompt}\n\n**Assistant:**\n\n{fill(text, width=wrap_width)}"
    display(Markdown(md))

    # By default, do NOT return text to avoid Colab echo.
    if return_text:
        return text

In [ ]:
# Example prompt
run_chat("Explain edge computing and give one practical example.")

# You can reuse this cell with new prompts:
# run_chat("What is the difference between AI and machine learning?")
# run_chat("Summarize the history of quantum computers.")

**User:** Explain edge computing and give one practical example.

**Assistant:**

Edge computing is a distributed computing paradigm that brings data processing and storage closer to the location where data is generated, rather than relying solely on centralized cloud servers. By processing data at or near the "edge" of the network—such as on local devices, gateways, or nearby edge servers—edge computing reduces latency, conserves bandwidth, and enhances real-time decision-making.

**Practical Example:**  
In a smart factory, industrial sensors monitor machinery performance in real time. Instead of sending all sensor data to a distant cloud server for analysis, edge computing allows data to be processed locally on an edge device (like a smart gateway or industrial PC). This enables immediate detection of anomalies—such as overheating equipment—and triggers instant alerts or automatic shutdowns to prevent damage. This rapid response improves operational efficiency and safety without waiting for cloud-based processing.

In [ ]:
run_chat("describe sarcobesity,  how to get rid off it")

**User:** describe sarcobesity,  how to get rid off it

**Assistant:**

**Sarcobesity** is a term that combines "sarcopenia" (loss of muscle mass) and "obesity," describing a condition where individuals experience both significant muscle loss and excess body fat—particularly visceral fat. It reflects a paradoxical state often seen in aging populations or those with chronic illness: despite being overweight or obese, they suffer from reduced muscle strength and function due to muscle atrophy. This combination increases risks for frailty, metabolic dysfunction, poor mobility, and higher mortality.

---

### Understanding Sarcobesity  
- **Muscle Loss:** Reduced skeletal muscle mass and strength, often due to aging (sarcopenia), inactivity, malnutrition, or chronic disease.  
- **Excess Fat:** High body fat, especially abdominal fat, linked to insulin resistance, inflammation, and cardiovascular risks.  
- **Combined Impact:** The loss of muscle exacerbates obesity-related complications, while obesity can worsen muscle decline through inflammation and metabolic stress.

---

### How to Address and Reverse Sarcobesity  

1. **Nutrition**  
   - **Protein Intake:** Consume adequate high-quality protein (1.0–1.2 g/kg body weight daily; up to 1.5 g/kg if frail) from sources like lean meats, fish, eggs, legumes, and dairy to support muscle synthesis.  
   - **Balanced Diet:** Include fiber-rich vegetables, whole grains, healthy fats (e.g., nuts, olive oil), and limit processed sugars and refined carbs.  
   - **Caloric Balance:** Avoid excessive calorie intake but ensure sufficient energy to prevent muscle breakdown—moderate deficit with nutrient density is key.  

2. **Resistance Training**  
   - Engage in regular strength training (2–3 times weekly) using weights, resistance bands, or bodyweight exercises to stimulate muscle growth and improve strength.  
   - Focus on major muscle groups, progressing intensity gradually.  

3. **Aerobic Exercise**  
   - Incorporate moderate-intensity cardio (e.g., brisk walking, cycling) to support cardiovascular health and aid fat loss without compromising muscle.  

4. **Medical Supervision**  
   - Consult healthcare providers to rule out underlying conditions (e.g., diabetes, hypothyroidism) and tailor a plan.  
   - Monitor muscle mass and fat levels via tests like DEXA scans or bioelectrical impedance.  

5. **Lifestyle Adjust

In [ ]:
run_chat("How to address and reverse sarcopenia?", max_new_tokens=1024)

**User:** How to address and reverse sarcopenia?

**Assistant:**

Sarcopenia, the age-related loss of muscle mass and strength, can be effectively addressed and partially reversed through a combination of lifestyle interventions. Here’s a comprehensive approach:

### 1. **Resistance Training (Strength Training)**
- **Key intervention:** Regular resistance exercise is the most effective way to stimulate muscle growth.
- Aim for at least 2–3 sessions per week targeting major muscle groups.
- Exercises may include weight lifting, bodyweight exercises, resistance bands, or machines.
- Progressive overload (gradually increasing weight or intensity) is crucial for continued muscle adaptation.

### 2. **Adequate Protein Intake**
- Consume sufficient high-quality protein daily (1.0–1.2 g/kg body weight, or up to 1.5 g/kg for older adults).
- Distribute protein evenly across meals (20–30 grams per meal).
- Include sources like lean meats, fish, eggs, dairy, legumes, and plant-based proteins.

### 3. **Balanced Nutrition**
- Ensure overall caloric intake meets energy needs without excessive excess.
- Include sufficient vitamins and minerals—especially vitamin D, calcium, magnesium, and B vitamins—that support muscle function and repair.
- Stay hydrated.

### 4. **Aerobic Exercise**
- Moderate-intensity aerobic activity (e.g., brisk walking, cycling) supports cardiovascular health and complements strength training by improving endurance and metabolic function.

### 5. **Hormonal and Medical Considerations**
- Assess and manage underlying conditions (e.g., hypogonadism, thyroid disorders) that may contribute to muscle loss.
- In some cases, medical evaluation may identify need for hormone therapy (e.g., testosterone in select patients), but this should be supervised by a healthcare provider.

### 6. **Lifestyle Factors**
- Avoid prolonged inactivity and sedentary behavior.
- Ensure adequate sleep for recovery and muscle synthesis.
- Manage chronic inflammation and stress, which can accelerate muscle degradation.

---

**Summary:**  
Reversing sarcopenia requires consistent resistance training, optimized nutrition with sufficient protein, and supportive lifestyle habits. Early intervention yields the best results, so starting as soon as possible after diagnosis is key. Consulting healthcare professionals—including a physician, physical therapist, and dietitian—ensures a personalized and safe plan.

In [ ]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace Copy_of_LFM2_MoE_Inference.ipynb

[NbConvertApp] WARNING | pattern 'Copy_of_LFM2_MoE_Inference.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True